<a href="https://colab.research.google.com/github/MNoichl/tttms_public/blob/main/text_layout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependencies

In [ ]:
!pip install metaknowledge
!pip install umap-learn
!pip install hdbscan
!pip install faerun
!pip install colorgram
!pip install colorthief
!pip install colorutils 
!pip install cylouvain
!pip install xmltodict
!pip install flatten_dict
!pip install boto3
!pip install compress-pickle
!pip install kaggle
!pip install compress-pickle

# Loading packages

In [ ]:

import metaknowledge as mk
import pandas as pd
import numpy as np
from random import randint
import datetime
import copy
from sklearn.cluster import KMeans
import shelve

import umap
from scipy import stats

from tqdm import tqdm_notebook as tqdm


from scipy.sparse import coo_matrix, vstack
from scipy.sparse import csr_matrix
import scipy as scipy

import math
%matplotlib inline

import seaborn as sns
import matplotlib.pyplot as plt

#For Tables:
from IPython.display import display
from IPython.display import Latex
pd.set_option('display.max_columns', 500)

#For R (ggplot2)
%load_ext rpy2.ipython

# from sklearn.externals.joblib import Memory
# memory = Memory(cachedir='/tmp', verbose=0)
# @memory.cache

import gc
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.cluster import adjusted_rand_score, mutual_info_score

import re
from joblib import Parallel, delayed
import multiprocessing
#Embedding:
#Clustering:
import hdbscan
from sklearn.decomposition import TruncatedSVD


import colorcet as cc
from matplotlib import colors
from faerun import Faerun, host
import os


import nltk
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from nltk import WordPunctTokenizer
import re
from nltk.stem import WordNetLemmatizer

from multiprocessing.dummy import Pool  # This is a thread-based Pool
from multiprocessing import cpu_count
import time

import networkx as nx

# import colorgram
from colorthief import ColorThief
from colorutils import Color,hex_to_hsv,hsv_to_hex,rgb_to_hex
import cylouvain

import pickle
import matplotlib as mpl

from compress_pickle import dump, load
import zipfile

# Loading Data 
(This doesn't need to run for re-runs: Just load the whole data below)


## Arxiv Data

In [ ]:
# Delete before publication !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!



!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"kaggle_username","key":"kaggle_private_key"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download Cornell-University/arxiv

In [ ]:
import zipfile
with zipfile.ZipFile('arxiv.zip', 'r') as zip_ref:
    zip_ref.extractall('arxiv_metadata')

In [ ]:
selected_years = ['2019','2020']
json_lines = []
for line in tqdm(open(r'arxiv_metadata/arxiv-metadata-oai-snapshot.json', 'r')):
  my_dict = json.loads(line)
  if re.findall('\d{4}',my_dict['versions'][0]['created'])[0] in selected_years:
    json_lines.append(my_dict)#json.loads(line))

arxiv_data = pd.DataFrame(json_lines)
del json_lines


## BiorXiv

In [ ]:
!gdown https://drive.google.com/uc?id=1hO6iNDVNJwevDLKQXJoqqiKZN9Y3wVCK

In [ ]:
biorxiv_df = pd.read_pickle("full_frame.pkl")  

In [ ]:
biorxiv_df

In [ ]:
biorxiv_df_redux = biorxiv_df[["published",	"title", 	"authors",	"date", "abstract", "category","doi"]]

In [ ]:
arxiv_data_redux = arxiv_data[["doi",	"title", 	"authors",	"update_date", "abstract", "categories","id"]]
arxiv_data_redux.columns = ["doi",	"title", 	"authors",	"date", "abstract", "category","id"]
biorxiv_df_redux.columns = ["doi",	"title", 	"authors",	"date", "abstract", "category","id"]

biorxiv_df_redux['origin'] = 'biorxiv'
arxiv_data_redux['origin'] = 'arxiv'

full_data = pd.concat([biorxiv_df_redux,arxiv_data_redux])

In [ ]:
full_data =full_data.drop_duplicates(subset=['id'], keep='last').reset_index(drop=True)

# Load full-data from drive

In [ ]:
!gdown --fuzzy https://drive.google.com/file/d/1-5ZrFDj_6tff7tCRd8nqlVf6Ls-l8-x1/view?usp=sharing -O "full_data_joined_with_formulas.bz"
full_data = load("full_data_joined_with_formulas.bz")

# Prepare texts

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
en_stop =set(stopwords.words('english'))
stemmer = WordNetLemmatizer()

def preprocess_text(document):
        # Remove all the special characters and numbers
        document = re.sub(r'\W|\d', ' ', str(document))

        # remove all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

        # Remove single characters from the start
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)

        # Substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)

        # Removing prefixed 'b'
        document = re.sub(r'^b\s+', '', document)

        # Converting to Lowercase
        document = document.lower()

        # Lemmatization
        tokens = document.split()
        tokens = [stemmer.lemmatize(word) for word in tokens]
#         tokens = [word for word in tokens if word not in en_stop]
        tokens = [word for word in tokens if len(word) > 3]

        preprocessed_text = ' '.join(tokens)

        return preprocessed_text

    

In [ ]:
df = full_data['abstract']
pool = Pool(8)
results = list(tqdm(pool.imap(preprocess_text, df))) # this does need to be restarted twice!


In [ ]:

vec = TfidfVectorizer(min_df=15,
                     norm=None,
                      use_idf=True, smooth_idf=True, sublinear_tf=True, max_df=0.4)
bow = vec.fit_transform(results)
SVD = TruncatedSVD(n_components= 100, n_iter=7, random_state=42)
XSVD = SVD.fit_transform(bow)
print(pd.DataFrame(bow.sum(axis=0),columns=vec.get_feature_names()).T[0].sort_values())


# Compute the umap-embedding

In [ ]:
import umap
from scipy import stats
to_cluster_umap = umap.UMAP(random_state=20,
                    n_components=2,
                    n_neighbors=20,
                    min_dist=0.1,
                    metric='cosine',
                    verbose=True,
                    low_memory=True)

to_cluster_umap.fit(XSVD)
to_cluster = to_cluster_umap.embedding_

# Cluster the k-nn-graph of the umap-embedding

In [ ]:
G = nx.from_scipy_sparse_matrix(to_cluster_umap.graph_)


In [ ]:
#  we run a clustering solution at high resolution, to produce a coherent color scheme.
macro_clustering = cylouvain.best_partition(G, resolution = 5)
macro_clustering_solution = list(macro_clustering.values())
macro_col_len = len(set(macro_clustering_solution))
print(macro_col_len)#6

In [ ]:
clustering = cylouvain.best_partition(G, resolution = 1.)
clustering_solution = list(clustering.values())
col_len = len(set(clustering_solution))
print(col_len)#25

In [ ]:


from collections import Counter

macro_cluster_counter = dict(Counter(macro_clustering_solution))

macro_clustering_solution_with_noise = []
for assignement in macro_clustering_solution:
  if macro_cluster_counter[assignement]<2000:
    macro_clustering_solution_with_noise.append(np.nan)
  else:
    macro_clustering_solution_with_noise.append(assignement)
macro_clustering_solution_with_noise = pd.factorize(macro_clustering_solution_with_noise, na_sentinel =-1)[0]


cluster_counter = dict(Counter(clustering_solution))

clustering_solution_with_noise = []
for assignement in clustering_solution:
  if cluster_counter[assignement]<1000:
    clustering_solution_with_noise.append(np.nan)
  else:
    clustering_solution_with_noise.append(assignement)
clustering_solution_with_noise = pd.factorize(clustering_solution_with_noise, na_sentinel =-1)[0]




In [ ]:

combination_counter = dict(Counter(list(zip(macro_clustering_solution_with_noise,clustering_solution_with_noise))))

color_combinations={}
for this_cluster in np.unique(clustering_solution_with_noise):
  in_these_combinations = []
  for this_co in combination_counter.keys():
    if this_co[1] == this_cluster:
      in_these_combinations.append(this_co)
  # print(in_these_combinations)
  combo= in_these_combinations[np.argmax([combination_counter[x] for x in in_these_combinations])]
  color_combinations[combo[1]] = combo[0]


In [ ]:
print(len(np.unique(macro_clustering_solution_with_noise)))
macro_cluster_colors = ['#b3c3b9','#b9e26f','#4e92ba','#ff8c02','#d15a0a','#541836']

sns.palplot(macro_cluster_colors)




In [ ]:
import colorsys

def hex2rgb(hex_value):
    h = hex_value.strip("#") 
    rgb = tuple(int(h[i:i+2], 16) for i in (0, 2, 4))
    return [x/255 for x in rgb]

def return_jittered_color(my_color):
  hls = colorsys.rgb_to_hls(*hex2rgb(my_color))
  jitter = [(np.random.rand()-.5)*.05,
            (np.random.rand()-.5)*.2,
            (np.random.rand()-.5)*.2]
  hls = [abs(x+y)  if abs(x+y)<1 else 1. for x,y in zip(hls,jitter)]
  return colorsys.hls_to_rgb(*hls)

def return_jittered_color_rgb(my_color):
  rgb = hex2rgb(my_color)
  jitter = [np.random.rand()*.2,
            np.random.rand()*.2,
            np.random.rand()*.2]
  rgb = [abs(x+y)  if abs(x+y)<1 else 1. for x,y in zip(rgb,jitter)]
  return rgb


# return_jittered_color('#b3c3b9')
micro_colors = [macro_cluster_colors[color_combinations[x] + 1] for x in np.unique(clustering_solution_with_noise)]
sns.palplot(micro_colors)
micro_colors = [return_jittered_color(x) for x in micro_colors]
print(micro_colors)
sns.palplot(micro_colors)
colors_to_plot = [micro_colors[x+1] for x in clustering_solution_with_noise]

# Plot, colored by source

In [ ]:
import palettable
origin_colormap = palettable.wesanderson.GrandBudapest4_5.mpl_colormap


origin_factors = pd.factorize(full_data['origin'])
plot_idx = np.random.permutation(full_data.shape[0])

sns.set(font="Source Sans Pro",font_scale=2.1)
sns.set_style("white")

fig, ax = plt.subplots(figsize=(30,30))
scatter = plt.scatter(to_cluster[plot_idx, 0],
                      to_cluster[plot_idx, 1],
                      # s=[len(x)*4 for x in list(combined_dataset['article_tuple_lists'])],
                      c=origin_factors[0][plot_idx],
                      cmap=origin_colormap, alpha=1,s=1.5)

# plt.scatter(umapped_articles[:, 0], umapped_articles[:, 1],
#             s= [len(x)*4 for x in list(combined_dataset['article_tuple_lists'])]
#             , c=combined_dataset['origin'][plot_idx].factorize()[0],alpha=0.2)

plt.legend(handles=scatter.legend_elements()[0],
           labels=['BioRxiv','ArXiv'],markerscale=4)
scatter.set_alpha(.2) #set alpha after plotting the legend ;-)


plt.axis('equal')
plt.savefig('combined_set_ArX_BioRx.png', dpi=300)
plt.show()

# Plot, colored by cluster

In [ ]:
import colorcet as cc

sns.set(font="Source Sans Pro",font_scale=2.1)
sns.set_style("white")

fig, ax = plt.subplots(figsize=(30,30))
scatter = plt.scatter(to_cluster[:, 0],
                      to_cluster[:, 1],
                      c=colors_to_plot, alpha=.2,s=1.5)


plt.axis('equal')
plt.savefig('combined_set_cluster.png', dpi=300)
plt.show()

# Extract labels for clusters

In [ ]:
import textwrap

article_cluster_labels = []
for cluster_num in tqdm(np.unique(clustering_solution_with_noise)):
    w = np.where(np.array(clustering_solution_with_noise)==cluster_num)[0]
    cluster_vectors = bow[w]
    kws = [vec.get_feature_names()[i] for i in np.asarray(cluster_vectors.sum(axis=0)).flatten().argsort()[-12:]][::-1]
    article_cluster_labels.append(', '.join(kws))
     
article_cluster_labels_wrapped = ["\n".join(textwrap.wrap(i,60)) for i in article_cluster_labels]


In [ ]:
try:
  !rm cluster_labels.txt
except:
  pass
  
for cluster in article_cluster_labels:
  print(cluster)
  with open('drive/MyDrive/ARXIV_FORMULA_PARSING/cluster_labels.txt', 'a') as f:
    f.write(cluster)
    f.write('\n')

In [ ]:
full_data['cluster'] = clustering_solution_with_noise
full_data['color'] = colors_to_plot



dump(full_data, "drive/MyDrive/ARXIV_FORMULA_PARSING/full_data_rejoined_with_clusters.bz")

# Plot cluster-labels as base for final graphic

In [ ]:

sns.set(font="Source Sans Pro",font_scale=2.1)

x_dim = 4
sns.set_style("white")

cluster_vector = list(range(0,len(np.unique(clustering_solution_with_noise))))
cluster_centers=[]
for small_clust in np.unique(clustering_solution_with_noise):
    where_small_clust = np.where(np.array(clustering_solution_with_noise)==small_clust)[0]
    cluster_centers.append(np.median(to_cluster[where_small_clust],axis=0))
cluster_centers = np.array(cluster_centers)


fig, ax = plt.subplots(figsize=(40,40))


plt.scatter(to_cluster[:, 0], to_cluster[:, 1], s=10, c=colors_to_plot)
 


plot_dimensions = [np.min(to_cluster[:, 0]),np.max(to_cluster[:, 0]),np.min(to_cluster[:, 1]),np.max(to_cluster[:, 1])]
print(plot_dimensions)

x_label_size = 3
y_label_size = 1

plot_width = plot_dimensions[1] - plot_dimensions[0]
plot_height = plot_dimensions[3] - plot_dimensions[2]

margin_size = 400


left_angles = [i*(math.pi*2) for i in list(np.linspace(0.35,0.65,num=int(np.ceil(len(cluster_centers) / 2))))] 

right_angles = [i*(math.pi*2) for i in list(np.linspace(0.85,1.15,num=int(np.floor(len(cluster_centers) / 2))))] 


angle = left_angles + right_angles
print(angle)

circle_x = [(math.cos(i)*plot_width/1.2 + (plot_dimensions[0]+plot_width/2))for i in angle]
circle_y = [math.sin(i)*plot_height/1.9 + (plot_dimensions[2]+plot_height/2) for i in angle]




from scipy.optimize import linear_sum_assignment
from scipy.spatial import distance_matrix

euclidean_cost_matrix = distance_matrix(cluster_centers, np.array([circle_x,circle_y]).transpose())
row_ind, col_ind = linear_sum_assignment(euclidean_cost_matrix)


horizontal_constant = 0.66
marker_text_move = 0.5
for i in range(len(cluster_centers)):
    

    plt.scatter(circle_x[col_ind[i]],circle_y[col_ind[i]],s=300,c=np.array([micro_colors[i]]),marker='s')#,cmap=cmap)


    if circle_x[col_ind[i]] < plot_dimensions[0]+plot_width/2: #check if left
        dist = (cluster_centers[i,0] - circle_x[col_ind[i]]) * horizontal_constant

        ax.plot([circle_x[col_ind[i]],circle_x[col_ind[i]]+dist],
           [circle_y[col_ind[i]],circle_y[col_ind[i]]],
                c='black',linestyle=':',linewidth=3)
        
        ax.plot([circle_x[col_ind[i]]+dist,cluster_centers[i,0]],
           [circle_y[col_ind[i]],cluster_centers[i,1]],
                c='black',linestyle=':',linewidth=3)        

        
        ax.text(circle_x[col_ind[i]]-marker_text_move,circle_y[col_ind[i]], article_cluster_labels_wrapped[i],
           verticalalignment='center', horizontalalignment='right',style='italic')
        
        
    else:
        
        dist = (circle_x[col_ind[i]] - cluster_centers[i,0]) * horizontal_constant

        ax.plot([circle_x[col_ind[i]],circle_x[col_ind[i]]-dist],
           [circle_y[col_ind[i]],circle_y[col_ind[i]]],
                c='black',linestyle=':',linewidth=3)
        
        ax.plot([circle_x[col_ind[i]]-dist,cluster_centers[i,0]],
           [circle_y[col_ind[i]],cluster_centers[i,1]],
                c='black',linestyle=':',linewidth=3)     
        
        
        ax.text(circle_x[col_ind[i]]+marker_text_move,circle_y[col_ind[i]], article_cluster_labels_wrapped[i],
           verticalalignment='center', horizontalalignment='left',style='italic')
plt.axis('off')



plt.savefig('labeled_graphic.png', dpi=300)

# Plot overview graphic with varying settings

In [ ]:

from umap.umap_ import nearest_neighbors



n_neighbors = [5, 20,35,50]
min_dists = [0, 0.1,0.2,0.3]

precomputed_knn_embeddings = np.zeros((4, 4, len(XSVD), 2))

_knn = nearest_neighbors(XSVD,
                              n_neighbors=np.max(n_neighbors),
                              metric="cosine",
                              metric_kwds=None,
                              angular=False,
                              random_state=None,verbose=True
                             )


In [ ]:
for i, k in enumerate(n_neighbors):
    for j, dist in enumerate(min_dists):
        precomputed_knn_embeddings[i, j] = umap.UMAP(n_neighbors=k,
                                                      min_dist=dist,
                                                      precomputed_knn=_knn,verbose=True
                                                      ).fit_transform(XSVD)

In [ ]:
from compress_pickle import dump, load
fname = "drive/MyDrive/preprint_2019-2020_datasets/precomputed_knn_embeddings.bz" 
dump(precomputed_knn_embeddings, fname)

In [ ]:
fig, axs = plt.subplots(4, 4, figsize=(30,30))

for i, ax_row in enumerate(axs):
    for j, ax in enumerate(ax_row):
        ax.scatter(precomputed_knn_embeddings[i, j, :, 0],
                   precomputed_knn_embeddings[i, j, :, 1],
                  #  c=labels / 9,
                  #  cmap='tab10',
                   c=colors_to_plot,
                   alpha=0.1,
                   s=.8,
                   )
        ax.set_xticks([])
        ax.set_yticks([])
        if i == 0:
            ax.set_title("min_dist = {}".format(min_dists[j]), size=15)
        if j == 0:
            ax.set_ylabel("n_neighbors = {}".format(n_neighbors[i]), size=15)
fig.suptitle("UMAP embedding of MNIST digits with grid of parameters", y=0.92, size=20)
plt.subplots_adjust(wspace=0.05, hspace=0.05)

plt.savefig('overview_of_textual_umap_runs.png', dpi=300)#, ext='png', bbox_inches="tight")  


# Calculate progress graphics for illustration


In [ ]:
reducer = umap.UMAP(random_state=20,
                    n_components=2,
                    n_neighbors=20,
                    min_dist=0.1,
                    metric='cosine',
                    verbose=True,
                    low_memory=True,
                    n_epochs=0)

import palettable

init = XSVD[:,0:2]

for n_e in [0,1,2,5,10,20,50,100,200]:

  reducer = umap.UMAP(random_state=20,
                    n_components=2,
                    n_neighbors=20,
                    min_dist=0.01,
                    metric='cosine',
                    verbose=True,
                    low_memory=True,
                    n_epochs=n_e, init = init)
  

  embedding = reducer.fit_transform(XSVD)

  origin_factors = pd.factorize(full_data['origin'])
  plot_idx = np.random.permutation(full_data.shape[0])

  
  origin_colormap = palettable.wesanderson.GrandBudapest4_5.mpl_colormap

  plot_idx = np.random.permutation(embedding.shape[0])

  sns.set(font="Source Sans Pro",font_scale=2.1)
  sns.set_style("white")
  fig, ax = plt.subplots(figsize=(30,20))
  scatter = plt.scatter(embedding[plot_idx, 0],
                        embedding[plot_idx, 1],
                        s=1.5,
                        c=origin_factors[0][plot_idx],
                        cmap=origin_colormap,
                         alpha=1)
  

  plt.legend(handles=scatter.legend_elements()[0], labels=['ArXiv', 'BioRxiv'],markerscale=4)
  scatter.set_alpha(0.5) #set alpha after plotting the legend ;-)


  plt.axis('equal')
  plt.savefig('combined_set_ArX_BioRx_ne_'+str(n_e)+'.png', dpi=300)
  plt.show()